In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/Db-IMDB-Assignment.db")

result = pd.read_sql_query("SELECT * FROM MOVIE LIMIT 10 ", conn)
result


,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529
4,4,tt8239946,Tumbbad,2018,8.5,7483
5,5,tt7027278,Kedarnath,2018,5.5,1970
6,6,tt3498820,Captain America: Civil War,2016,7.8,536641
7,7,tt8108198,Andhadhun,2018,9.0,18160
8,8,tt3741834,Lion,2016,8.1,170216
9,9,tt6747420,Rajma Chawal,2018,5.7,681


1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that
the genre is 'Comedy’ and year is a leap year) Your query should return director name, the
movie name, and the year.

In [ ]:
res = pd.read_sql_query(
                        "SELECT  \
                          DISTINCT TRIM(P.Name) As Director_Name\
                        ,M.title As Movie_Name \
                        ,M.year AS Year \
                        FROM M_GENRE AS M_G \
                        LEFT JOIN GENRE AS G \
                        ON M_G.GID = G.GID \
                        ,    Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Director As MD \
                        ON M.MID = MD.MID AND P.PID = MD.PID \
                        WHERE (Trim(M.year) %4 ==0) \
                        AND M.MID = M_G.MID AND Trim(G.NAME) LIKE '%COMEDY%' ", conn)
res

,Director_Name,Movie_Name,Year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008
...,...,...,...
241,Siddharth Anand Kumar,Let's Enjoy,2004
242,Amma Rajasekhar,Sathyam,2008
243,Oliver Paulus,Tandoori Love,2008
244,Raja Chanda,Le Halua Le,2012


2. List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
res1 = pd.read_sql_query(
                        "SELECT  \
                        P.Name As Actor_Name\
                        FROM Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Cast As C \
                        ON TRIM(M.MID) = TRIM(C.MID) \
                        AND TRIM(P.PID) = TRIM(C.PID) \
                        WHERE M.title LIKE 'Anand' ", conn)
res1

,Actor_Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970
and > 1990.)

In [ ]:
#updated as suggested

Actors_1970_1990 = pd.read_sql_query("""

                                    WITH
                                    
                                    Actors_Before_1970 AS
                                    (SELECT DISTINCT TRIM(MC.PID) PID
                                    FROM MOVIE M
                                    JOIN M_CAST MC ON M.MID = MC.MID
                                    WHERE CAST(SUBSTR(M.year,-4) AS UNASSIGNED) < 1970),
                                    
                                    Actors_After_1990 AS
                                    (SELECT DISTINCT TRIM(MC.PID) PID
                                    FROM MOVIE M
                                    JOIN M_CAST MC ON M.MID = MC.MID
                                    WHERE CAST(SUBSTR(M.year,-4) AS UNASSIGNED) > 1990),
                                    
                                    ACTORS AS
                                    (SELECT A_1970.PID PID
                                    FROM Actors_Before_1970 A_1970
                                    JOIN Actors_After_1990 A_1990 ON A_1970.PID = A_1990.PID)
                                    
                                    SELECT DISTINCT TRIM(P.NAME) Actors_Before_1970_After_1990
                                    FROM PERSON P
                                    JOIN ACTORS A ON A.PID = TRIM(P.PID)
                                    
                                    """, conn)

In [ ]:
print("\nActors who worked in movies before 1970 and after 1990:")
print('-'*55, '\n')
print(Actors_1970_1990)


Actors who worked in movies before 1970 and after 1990:
------------------------------------------------------- 

    Actors_Before_1970_After_1990
0                    Rishi Kapoor
1                Amitabh Bachchan
2                          Asrani
3                    Zohra Sehgal
4                 Parikshat Sahni
..                            ...
295                        Poonam
296                 Jamila Massey
297                   K.R. Vijaya
298                         Sethi
299                  Suryakantham

[300 rows x 1 columns]


4. List all directors who directed 10 movies or more, in descending order of the number of
movies they directed. Return the directors' names and the number of movies each of them
directed.

In [ ]:
res = pd.read_sql_query(
                        "SELECT  \
                              TRIM(P.Name)       As Director_Name \
                              ,COUNT(M.MID)       As Movie_COUNT \
                        FROM Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Director As MD \
                        ON M.MID = MD.MID \
                        AND P.PID = MD.PID \
                        GROUP BY TRIM(P.NAME) \
                        HAVING COUNT(M.MID) >= 10 \
                        ORDER BY COUNT(M.MID) DESC ", conn)
res

,Director_Name,Movie_COUNT
0,David Dhawan,78
1,Mahesh Bhatt,72
2,Ram Gopal Varma,60
3,Vikram Bhatt,58
4,Hrishikesh Mukherjee,54
...,...,...
151,Siddharth Anand,10
152,Subhash Kapoor,10
153,Subodh Mukherji,10
154,Sujoy Ghosh,10


5.

a. For each year, count the number of movies in that year that had only female actors.


In [ ]:


sql_query='''SELECT  distinct substr(year,-4) as 'Year', count(substr(year,-4)) AS 'No of Movies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and m.mid not in ( SELECT distinct m.mid FROM Movie m, M_Cast c, Person p 
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and p.gender = 'Male'   )
                        Group by substr(year,-4) ;   '''

result = pd.read_sql_query( sql_query, conn)
result


,Year,No of Movies
0,1939,1
1,1999,11
2,2000,10
3,2018,3


b. Now include a small change: report for each year the percentage of movies in that
year with only female actors, and the total number of movies made that year. For
example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were
13,522 movies, and 31.81% had only female actors. You do not need to round your

answer

In [ ]:
sql_query='''Select distinct m.year as 'Year',m.TMovies as 'Total Movies', ((a.FMovies * 100.00) / m.TMovies) as '%age of Female Movies'
                from  (SELECT substr(m.year,-4) as 'Year', count(m.year) AS 'TMovies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID \
                        and trim(c.PID) = p.PID \
                        Group by substr(m.year,-4)) m\
                Join ( SELECT  distinct substr(m.year,-4) as 'Year', count(substr(year,-4)) AS 'FMovies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and m.mid not in ( SELECT distinct m.mid FROM Movie m, M_Cast c, Person p 
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and p.gender = 'Male'   )
                        Group by substr(m.year,-4)) a on m.year=a.year
                        
                group by m.year;   '''

result = pd.read_sql_query( sql_query, conn)
result


,Year,Total Movies,%age of Female Movies
0,1939,45,2.222222
1,1999,1628,0.675676
2,2000,1480,0.675676
3,2018,2942,0.101971


6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast
size" we mean the number of distinct actors that played in that movie: if an actor played
multiple roles, or if it simply occurs multiple times in casts, we still count her/him only
once.

In [ ]:
res1 = pd.read_sql_query("SELECT\
                            J1.MOVIE_TITLE \
                            ,MAX(J1.CAST_SIZE) AS CAST_SIZE \
                            FROM \
                            (SELECT  \
                            COUNT(DISTINCT C.PID) As CAST_SIZE \
                            ,TRIM(M.TITLE) AS MOVIE_TITLE \
                            FROM Movie As M \
                            LEFT JOIN M_Cast As C \
                            ON M.MID = C.MID \
                            GROUP BY TRIM(M.TITLE) ) J1\
     ", conn)
res1

,MOVIE_TITLE,CAST_SIZE
0,Ocean's Eight,238


7. A decade is a sequence of 10 consecutive years. For example, say in your database you have
movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the
second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of
films and the total number of films in D.

In [ ]:
Decade_More_Movies = pd.read_sql_query("""


                                        WITH 
            
                                        YEARS_UNIQUE AS
                                       (SELECT DISTINCT
                                        CAST(SUBSTR(year,-4) AS UNSIGNED) YEAR,
                                        CAST(SUBSTR(year,-4) AS UNSIGNED) DECADE_START,
                                        CAST(SUBSTR(year,-4) AS UNSIGNED) + 9 DECADE_END,
                                        'Decade of : ' || SUBSTR(year,-4) DECADE
                                        FROM MOVIE),
    
                                        MOVIE_COUNT_YEARS AS
                                       (SELECT COUNT(DISTINCT MID) Movie_Count, CAST(SUBSTR(year,-4) AS UNSIGNED) Year
                                        FROM MOVIE
                                        GROUP BY CAST(SUBSTR(year,-4) AS UNSIGNED)),
    
                                        MOVIE_COUNT_DECADE AS 
                                       (SELECT SUM(Movie_Count) Total_Movies, YU.DECADE
                                        FROM MOVIE_COUNT_YEARS MCY, YEARS_UNIQUE YU
                                        WHERE MCY.YEAR BETWEEN YU.DECADE_START AND YU.DECADE_END
                                        GROUP BY YU.DECADE)
    
                                        SELECT Decade, Total_Movies
                                        FROM MOVIE_COUNT_DECADE
                                        WHERE Total_Movies = (SELECT MAX(Total_Movies)
                                        FROM MOVIE_COUNT_DECADE)
                                        
                                        """, conn)

In [ ]:
print("\nMore movies in decade:")
print('-'*22, '\n')
print(Decade_More_Movies)


More movies in decade:
---------------------- 

             DECADE  Total_Movies
0  Decade of : 2008          1205


8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume
that the actors remain unemployed between two consecutive movies).


In [ ]:
Actors_Never_Unemployed_More_3 = pd.read_sql_query("""

                        WITH
    
                        ACTORS_MOVIE_YEAR AS
                        (SELECT TRIM(MC.PID) PID, CAST(SUBSTR(year,-4) AS UNASSIGNED) Year,
                        COUNT(DISTINCT TRIM(M.MID)) Number_of_Mov
                        FROM M_CAST MC, MOVIE M
                        WHERE TRIM(MC.MID) = TRIM(M.MID) 
                        GROUP BY TRIM(MC.PID), CAST(SUBSTR(year,-4) AS UNASSIGNED)
                        ORDER BY NUMBER_OF_MOV DESC),
    
                        ACTORS_MORE_THAN_YEAR AS
                        (SELECT AMY.PID, COUNT(AMY.YEAR) AS Number_of_Years, MIN(AMY.YEAR) AS Min_Year,
                        MAX(AMY.YEAR) AS Max_Year
                        FROM ACTORS_MOVIE_YEAR AMY
                        GROUP BY AMY.PID
                        HAVING COUNT(AMY.YEAR) > 1),
    
                        ACTORS_NUMBER_MORE_THAN_YEAR AS
                        (SELECT AMY.PID, AMY.YEAR, AMY.YEAR+4 AS Year_4, AMY.NUMBER_OF_MOV,
                        ATY.MIN_YEAR, ATY.MAX_YEAR
                        FROM ACTORS_MOVIE_YEAR AMY, ACTORS_MORE_THAN_YEAR ATY
                        WHERE AMY.PID = ATY.PID),
    
                        NUMBER_MOVIE_PRESENT AS
                        (SELECT AM.PID, NY.YEAR, SUM(AM.NUMBER_OF_MOV) AS NUMBER_OF_MOVIE_PRESENT
                        FROM ACTORS_NUMBER_MORE_THAN_YEAR AM, ACTORS_NUMBER_MORE_THAN_YEAR NY
                        WHERE AM.PID = NY.PID AND
                        AM.YEAR BETWEEN NY.MIN_YEAR AND NY.YEAR
                        GROUP BY AM.PID, NY.YEAR),
    
                        ACTOR_MOVIE_4_YEAR AS
                        (SELECT AM.PID, NY.YEAR, SUM(AM.NUMBER_OF_MOV) AS ACTOR_MOVIE_4_YEARS_PRESENT
                        FROM ACTORS_NUMBER_MORE_THAN_YEAR AM, ACTORS_NUMBER_MORE_THAN_YEAR NY
                        WHERE AM.PID = NY.PID AND
                        AM.YEAR BETWEEN NY.MIN_YEAR AND NY.YEAR_4 AND
                        NY.YEAR_4 <= NY.MAX_YEAR
                        GROUP BY AM.PID, NY.YEAR)
    
                        SELECT DISTINCT TRIM(P.NAME) AS ACTORS_NEVER_UNEMPLOYED_MORE_THAN_3_YEARS
                        FROM PERSON P
                        WHERE TRIM(P.PID) NOT IN (SELECT DISTINCT NMP.PID
                        FROM NUMBER_MOVIE_PRESENT NMP, ACTOR_MOVIE_4_YEAR AM_4
                        WHERE NMP.PID = AM_4.PID AND
                        NMP.YEAR = AM_4.YEAR AND
                        NMP.NUMBER_OF_MOVIE_PRESENT = AM_4.ACTOR_MOVIE_4_YEARS_PRESENT)
                        
                        
                        """, conn)

In [ ]:
print("\nActors who were never unemployed for more than 3 years at stretch:")
print('-'*70, '\n')
print(Actors_Never_Unemployed_More_3)


Actors who were never unemployed for more than 3 years at stretch:
---------------------------------------------------------------------- 

      ACTORS_NEVER_UNEMPLOYED_MORE_THAN_3_YEARS
0                                Christian Bale
1                                Cate Blanchett
2                          Benedict Cumberbatch
3                                 Naomie Harris
4                                   Andy Serkis
...                                         ...
32580                            Deepak Ramteke
32581                              Kamika Verma
32582                        Dhorairaj Bhagavan
32583                              Nasir Shaikh
32584                              Adrian Fulle

[32585 rows x 1 columns]


9. Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
res1 = pd.read_sql_query(
            "SELECT DISTINCT T1.ACTOR_NAME FROM \
                              (SELECT P1.NAME AS ACTOR_NAME \
                              ,COUNT(M.MID) AS MOVIE_COUNT_Y \
                              ,P1.PID AS PID \
                              FROM Movie As M \
                              LEFT JOIN Person As P1 \
                              ,         Person As P2 \
                              ,         M_Cast As C \
                              ,         M_Director MD \
                              ON  M.MID = C.MID \
                              AND P1.PID = TRIM(C.PID) \
                              AND M.MID = MD.MID \
                              AND P2.PID = MD.PID \
                              WHERE  TRIM(P2.NAME) LIKE 'Yash Chopra' \
                              GROUP BY TRIM(P1.Name), P1.PID \
                              ) T1 \
                      JOIN \
                            (SELECT P1.NAME AS ACTOR_NAME \
                              ,P2.NAME AS DIRECTOR_NAME \
                              ,COUNT(M.MID) AS MOVIE_COUNT \
                              ,P1.PID AS PID \
                              FROM Movie As M \
                              LEFT JOIN Person As P1 \
                              ,         Person As P2 \
                              ,         M_Cast As C \
                              ,         M_Director MD \
                              ON  M.MID = C.MID \
                              AND P1.PID = TRIM(C.PID) \
                              AND M.MID = MD.MID \
                              AND P2.PID = MD.PID \
                              WHERE  TRIM(P2.NAME) NOT LIKE 'Yash Chopra' \
                              GROUP BY P1.Name, P2.Name, P1.PID \
                            ) T2 \
            ON T1.PID = T2.PID \
            AND T1.MOVIE_COUNT_Y  > T2.MOVIE_COUNT    ", conn)
res1

,ACTOR_NAME
0,A.K. Hangal
1,Aamir Khan
2,Achala Sachdev
3,Ajit
4,Akhilendra Mishra
...,...
335,Waheeda Rehman
336,Yash Chopra
337,Yashodra Katju
338,Yunus Parvez


10. The Shahrukh number of an actor is the length of the shortest path between the actor and
Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all
actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who
acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc.
Return all actors whose Shahrukh number is 2.

In [ ]:
res1 = pd.read_sql_query("\
                          SELECT DISTINCT P1.NAME \
                          FROM MOVIE AS M1 \
                          LEFT JOIN Person As P1 \
                          ,         M_Cast As C1 \
                          ON  M1.MID = C1.MID \
                          AND P1.PID = TRIM(C1.PID) \
                          AND M1.MID IN(\
                                        SELECT DISTINCT M1.MID \
                                        FROM MOVIE AS M1 \
                                        LEFT JOIN Person As P1 \
                                        ,         M_Cast As C1 \
                                        ON  M1.MID = C1.MID \
                                        AND P1.PID = TRIM(C1.PID) \
                                        AND P1.PID IN\
                                                      (\
                                                      SELECT DISTINCT P1.PID \
                                                      FROM MOVIE AS M1 \
                                                      LEFT JOIN Person As P1 \
                                                      ,         M_Cast As C1 \
                                                      ON  M1.MID = C1.MID \
                                                      AND P1.PID = TRIM(C1.PID) \
                                                      AND P1.PID NOT IN('nm0451321')\
                                                      WHERE M1.MID IN (\
                                                                      SELECT DISTINCT M1.MID \
                                                                      FROM MOVIE AS M1 \
                                                                      LEFT JOIN Person As P1 \
                                                                      ,         M_Cast As C1 \
                                                                      ON  M1.MID = C1.MID \
                                                                      AND P1.PID = TRIM(C1.PID)\
                                                                      WHERE TRIM(P1.NAME) LIKE '%Shah Rukh Khan%' \
                                                                        )\
                                                      )\
                                  )\
                                          ", conn)
res1

,Name
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
...,...
26877,Minoo Mehtab
26878,Hayley Cleghorn
26879,Nirvasha Jithoo
26880,Kamal Maharshi


In [ ]:
Source Links
https://www.appliedaicourse.com/
https://www.w3schools.com/sql/default.asp
https://www.w3resource.com/sql-exercises/movie-database-exercise/joins-exercises-on-movie-database.php
-

In [ ]:
Query_9 = pd.read_sql_query('''SELECT Distinct P1.Name,P1.PID,Count(Distinct M1.MID) as Num_Movie_wid_YC FROM Person P1 
                         Join M_Cast C on C.PID = P1.PID
                         Join Movie M1 on M1.MID = C.MID
                         Join M_Director D on D.MID = M1.MID
                         Join Person P2 on D.PID = P2.PID 
                         Where P2.Name = 'Yash Chopra'
                         Group By P1.PID
                         Having Num_Movie_wid_YC >
                         (Select  Count (Distinct M2.MID) From Movie M2
                         Join M_Cast C2 on C2.MID = M2.MID
                         Join Person P3 on C2.PID = P3.PID
                         Join M_Director D2 on D2.MID = M2.MID
                         Join Person P4 on D2.PID = P4.PID
                         Where P3.PID = P1.PID and NOT P4.Name = 'Yash Chopra' 
                         Group By P3.PID,D2.PID)
                         Order By Num_Movie_wid_YC DESC
                         ''',conn)

In [ ]:
sql_query='''select distinct a.name from ( 
                                select distinct p.pid,p.name,d.pid, count(*) as 'Total' from Movie m,M_Cast c,Person p , M_Director d
                                    where m.mid=c.mid
                                    and m.mid=d.mid
                                    and trim(c.pid)=p.pid
                                    and d.pid not in ('nm0007181')
                                    and p.pid in ( select distinct p.pid from Movie m,M_Cast c,Person p 
                                                where m.mid=c.mid
                                                and trim(c.pid)=p.pid  
                                                and m.mid in ( select m.mid from Movie m,M_Director d,Person p 
                                                                where m.mid=d.mid
                                                                and trim(d.pid)=p.pid  
                                                                and p.name like 'Yash%' )) group by p.pid,p.name,d.pid ) a 
                    
                            join (select distinct p.pid,count(trim(p.pid)) as 'Yash' from Movie m,M_Cast c,Person p 
                                    where m.mid=c.mid
                                    and trim(c.pid)=p.pid  
                                    and m.mid in ( select m.mid from Movie m,M_Director d,Person p 
                                                    where m.mid=d.mid
                                                    and trim(d.pid)=p.pid  
                                                    and p.name like 'Yash%' )
                                    group by p.pid) b on a.pid=b.pid
                            where  b.Yash > a.Total 
                            '''


In [ ]:
result = pd.read_sql_query( sql_query, conn)
result

,name


In [ ]:
pd.read_sql_query("select Director, Actor, Count(*) as Movies_with_YashChopra from(select p1.Name as Director, m1.title as Movie from Person p1 Inner Join M_Director md on TRIM(md.PID)=p1.PID Inner Join Movie m1 on TRIM(md.MID)=m1.MID and p1.Name LIKE 'Yash%' Group By p1.Name, m1.title) t1 Inner Join (select p2.Name as Actor,m2.title as Movie from Person p2 Inner Join M_Cast mc on TRIM(mc.PID)=p2.PID Inner Join Movie m2 on TRIM(mc.MID)=m2.MID Group By p2.Name, m2.title) t2 on t1.Movie=t2.Movie Group By t1.Director, t2.Actor Order By Movies_with_YashChopra DESC",conn)

,Director,Actor,Movies_with_YashChopra


In [ ]:
print(Query_9)

Empty DataFrame
Columns: [Name, PID, Num_Movie_wid_YC]
Index: []
